Netzwerk
========

__Lernziele__

-   Einen Überblick über Netzwerk-Konzepte haben
-   Anforderungen dafür verstehen, einen Linux-Rechner in ein lokales
    Netz zu integrieren
-   Wichtige Kommandos zur Fehlersuche kennen
-   Wichtige Netzwerkdienste kennen

__Vorkenntnisse__

-   Umgang mit Dateien und mit einem Texteditor
-   Kenntnisse über die Linux-Dateisystemstruktur
-   Vorkenntnisse im TCP/IP-Bereich und Kenntnisse im Umgang mit
    Netzwerkdiensten sind nützlich

* https://wiki.ubuntuusers.de/Netzwerk/

## Iproute2

Paket iproute2 (auch iproute)

- Ersetzen Klassiker ifconfig, arp, route
- Kernkomponente sind die Kommandos ip und tc

Siehe auch https://wiki.ubuntuusers.de/ip/

In [1]:
apt policy iproute2

iproute2:
  Installiert:           4.3.0-1ubuntu3.16.04.3
  Installationskandidat: 4.3.0-1ubuntu3.16.04.3
  Versionstabelle:
 *** 4.3.0-1ubuntu3.16.04.3 500
        500 http://us.archive.ubuntu.com/ubuntu xenial-updates/main amd64 Packages
        100 /var/lib/dpkg/status
     4.3.0-1ubuntu3 500
        500 http://us.archive.ubuntu.com/ubuntu xenial/main amd64 Packages


### Status aller Schnittstellen

In [118]:
ip link show

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN mode DEFAULT group default qlen 1
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
2: enp0s3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP mode DEFAULT group default qlen 1000
    link/ether 08:00:27:f2:c8:06 brd ff:ff:ff:ff:ff:ff
3: enp0s8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP mode DEFAULT group default qlen 1000
    link/ether 08:00:27:e7:7e:82 brd ff:ff:ff:ff:ff:ff
4: lxcbr0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc noqueue state UP mode DEFAULT group default qlen 1000
    link/ether 00:16:3e:00:00:00 brd ff:ff:ff:ff:ff:ff
5: docker0: <NO-CARRIER,BROADCAST,MULTICAST,UP> mtu 1500 qdisc noqueue state DOWN mode DEFAULT group default 
    link/ether 02:42:33:88:2c:04 brd ff:ff:ff:ff:ff:ff
6: virbr0: <NO-CARRIER,BROADCAST,MULTICAST,UP> mtu 1500 qdisc noqueue state DOWN mode DEFAULT group default qlen 1000
    link/ether 00:00:00:00:00:00 brd ff:ff:f

In [7]:
ip link show enp0s3

2: enp0s3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP mode DEFAULT group default qlen 1000
    link/ether 08:00:27:f2:c8:06 brd ff:ff:ff:ff:ff:ff


Links ansehen und manipulieren

-   UP → Schnittstelle aktiviert
-   LOWER\_UP und NO-CARRIER → Verbindung
-   BROADCAST und MULTICAST → beherrscht Interface
-   SLAVE → Interface gehört zu einem Bounding
-   POINTTOPOINT → Netzwerk Punkt zu Punkt

Option -s zeigt Statistik

In [11]:
ip -s link show enp0s3

2: enp0s3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP mode DEFAULT group default qlen 1000
    link/ether 08:00:27:f2:c8:06 brd ff:ff:ff:ff:ff:ff
    RX: bytes  packets  errors  dropped overrun mcast   
    127859367  98361    0       0       0       0       
    TX: bytes  packets  errors  dropped carrier collsns 
    3828892    58606    0       0       0       0       


Interface de- bzw. aktivieren

- \# ip link set eth0 down
- \# ip link set eth0 up

In [17]:
sudo ip link set enp0s3 down

In [18]:
ip link show enp0s3

2: enp0s3: <BROADCAST,MULTICAST> mtu 1500 qdisc pfifo_fast state DOWN mode DEFAULT group default qlen 1000
    link/ether 08:00:27:f2:c8:06 brd ff:ff:ff:ff:ff:ff


In [19]:
sudo ip link set enp0s3 up

In [24]:
ip link show enp0s3

2: enp0s3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP mode DEFAULT group default qlen 1000
    link/ether 08:00:27:f2:c8:06 brd ff:ff:ff:ff:ff:ff


MAC Adresse setzen

-   \# ip link set eth0 address 08:00:27:f2:af:fe

In [25]:
sudo ip link set enp0s3 address 08:00:27:f2:af:fe

In [26]:
ip link show enp0s3

2: enp0s3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP mode DEFAULT group default qlen 1000
    link/ether 08:00:27:f2:af:fe brd ff:ff:ff:ff:ff:ff


In [27]:
sudo ip link set enp0s3 address 08:00:27:f2:c8:06

Interfacename ändern

-   \# ip link set eth1 name extern

In [31]:
sudo ip link set enp0s3 down
sudo ip link set enp0s3 name vagrant
sudo ip link set vagrant up

Cannot find device "enp0s3"
Cannot find device "enp0s3"


In [32]:
ip link show vagrant

2: vagrant: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP mode DEFAULT group default qlen 1000
    link/ether 08:00:27:f2:c8:06 brd ff:ff:ff:ff:ff:ff


In [33]:
sudo ip link set vagrant down
sudo ip link set vagrant name enp0s3
sudo ip link set enp0s3 up

In [34]:
sudo systemctl restart networking.service

In [35]:
ip link show enp0s3

2: enp0s3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP mode DEFAULT group default qlen 1000
    link/ether 08:00:27:f2:c8:06 brd ff:ff:ff:ff:ff:ff


### IP-Adresse der Schnittstellen

In [36]:
ip address show

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
    inet6 ::1/128 scope host 
       valid_lft forever preferred_lft forever
2: enp0s3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 08:00:27:f2:c8:06 brd ff:ff:ff:ff:ff:ff
    inet 10.0.2.15/24 brd 10.0.2.255 scope global enp0s3
       valid_lft forever preferred_lft forever
    inet6 fe80::a00:27ff:fef2:c806/64 scope link 
       valid_lft forever preferred_lft forever
3: enp0s8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 08:00:27:e7:7e:82 brd ff:ff:ff:ff:ff:ff
    inet 172.22.3.2/24 brd 172.22.3.255 scope global enp0s8
       valid_lft forever preferred_lft forever
    inet6 fe80::a00:27ff:fee7:7e82/64 scope link 
       valid_lft f

In [37]:
ip address show enp0s8

3: enp0s8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 08:00:27:e7:7e:82 brd ff:ff:ff:ff:ff:ff
    inet 172.22.3.2/24 brd 172.22.3.255 scope global enp0s8
       valid_lft forever preferred_lft forever
    inet6 fe80::a00:27ff:fee7:7e82/64 scope link 
       valid_lft forever preferred_lft forever


- \# ip address add 192.168.99.14/24 broadcast 255.255.255.0 dev enp0s8  
  Immer boradcast (+ setzt alle Hostbits) und device angeben

In [49]:
sudo ip address add 192.168.99.14/24 broadcast 192.168.99.255 dev enp0s8

In [50]:
ip address show enp0s8

3: enp0s8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 08:00:27:e7:7e:82 brd ff:ff:ff:ff:ff:ff
    inet 172.22.3.2/24 brd 172.22.3.255 scope global enp0s8
       valid_lft forever preferred_lft forever
    inet 192.168.99.14/24 brd 192.168.99.255 scope global enp0s8
       valid_lft forever preferred_lft forever
    inet6 fe80::a00:27ff:fee7:7e82/64 scope link 
       valid_lft forever preferred_lft forever


-   \# ip address del 192.168.99.14/24 dev enp0s8

In [48]:
sudo ip address del 192.168.99.14/24 dev enp0s8

In [43]:
ip address show enp0s8

3: enp0s8: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 08:00:27:e7:7e:82 brd ff:ff:ff:ff:ff:ff
    inet 172.22.3.2/24 brd 172.22.3.255 scope global enp0s8
       valid_lft forever preferred_lft forever
    inet6 fe80::a00:27ff:fee7:7e82/64 scope link 
       valid_lft forever preferred_lft forever


-   \# ip -4 address flush label eth1 löscht alle IPv4

In [53]:
ip address show enp0s3

2: enp0s3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 08:00:27:f2:c8:06 brd ff:ff:ff:ff:ff:ff
    inet 10.0.2.15/24 brd 10.0.2.255 scope global enp0s3
       valid_lft forever preferred_lft forever
    inet6 fe80::a00:27ff:fef2:c806/64 scope link 
       valid_lft forever preferred_lft forever


In [54]:
sudo ip -4 address flush label enp0s3

In [55]:
ip address show enp0s3

2: enp0s3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 08:00:27:f2:c8:06 brd ff:ff:ff:ff:ff:ff


In [56]:
sudo ip address add 10.0.2.15/24 brd 10.0.2.255 dev enp0s3

In [57]:
ip address show enp0s3

2: enp0s3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 08:00:27:f2:c8:06 brd ff:ff:ff:ff:ff:ff
    inet 10.0.2.15/24 brd 10.0.2.255 scope global enp0s3
       valid_lft forever preferred_lft forever


#### Mehrere Adressen

- \# ip a a 192.168.200.1/24 brd + dev eth1  
  \# ip a a 192.168.200.2/24 brd + dev eth1  
  \# ip a a 192.168.200.3/24 brd + dev eth1  
  \# ip a a 192.168.200.4/24 brd + dev eth1
- \# ip a s eth1
- Erste Adresse ist die primäre Adresse. Alle anderen sind sekundäre Adressen.  
  Verlassen Pakete über eth1, bekommen diese die primäre Adresse. Jede neue Adresse aus einem anderen Netzbereich wird wiederum primäre Adresse.
- Löscht man eine primäre Adresse, werden auch sekundäre gelöscht.

In [65]:
sudo ip a a 192.168.200.1/24 brd 192.168.200.255 dev enp0s3

In [66]:
ip address show enp0s3

2: enp0s3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 08:00:27:f2:c8:06 brd ff:ff:ff:ff:ff:ff
    inet 10.0.2.15/24 brd 10.0.2.255 scope global enp0s3
       valid_lft forever preferred_lft forever
    inet 192.168.200.1/24 brd 192.168.200.255 scope global enp0s3
       valid_lft forever preferred_lft forever


In [67]:
sudo ip a a 192.168.200.2/24 brd 192.168.200.255 dev enp0s3
sudo ip a a 192.168.200.3/24 brd 192.168.200.255 dev enp0s3
sudo ip a a 192.168.200.4/24 brd 192.168.200.255 dev enp0s3

In [68]:
ip address show enp0s3

2: enp0s3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 08:00:27:f2:c8:06 brd ff:ff:ff:ff:ff:ff
    inet 10.0.2.15/24 brd 10.0.2.255 scope global enp0s3
       valid_lft forever preferred_lft forever
    inet 192.168.200.1/24 brd 192.168.200.255 scope global enp0s3
       valid_lft forever preferred_lft forever
    inet 192.168.200.2/24 brd 192.168.200.255 scope global secondary enp0s3
       valid_lft forever preferred_lft forever
    inet 192.168.200.3/24 brd 192.168.200.255 scope global secondary enp0s3
       valid_lft forever preferred_lft forever
    inet 192.168.200.4/24 brd 192.168.200.255 scope global secondary enp0s3
       valid_lft forever preferred_lft forever


In [69]:
sudo ip address del 192.168.200.1/24 dev enp0s3

In [70]:
ip address show enp0s3

2: enp0s3: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc pfifo_fast state UP group default qlen 1000
    link/ether 08:00:27:f2:c8:06 brd ff:ff:ff:ff:ff:ff
    inet 10.0.2.15/24 brd 10.0.2.255 scope global enp0s3
       valid_lft forever preferred_lft forever


### Route anzeigen

In [4]:
ip route show

default via 10.0.2.2 dev enp0s3 
10.0.2.0/24 dev enp0s3  proto kernel  scope link  src 10.0.2.15 
10.0.3.0/24 dev lxcbr0  proto kernel  scope link  src 10.0.3.1 
172.17.0.0/16 dev docker0  proto kernel  scope link  src 172.17.0.1 linkdown 
172.22.3.0/24 dev enp0s8  proto kernel  scope link  src 172.22.3.2 
192.168.122.0/24 dev virbr0  proto kernel  scope link  src 192.168.122.1 linkdown 


#### Routing Referenz 

In [76]:
ip r s

10.0.2.0/24 dev enp0s3  proto kernel  scope link  src 10.0.2.15 
10.0.3.0/24 dev lxcbr0  proto kernel  scope link  src 10.0.3.1 
172.17.0.0/16 dev docker0  proto kernel  scope link  src 172.17.0.1 linkdown 
172.22.3.0/24 dev enp0s8  proto kernel  scope link  src 172.22.3.2 
192.168.122.0/24 dev virbr0  proto kernel  scope link  src 192.168.122.1 linkdown 


-   prote kernel → Weg ging über Kernel Mehr /etc/iproute2/rt\_protos
-   scope link → Netz steht direkt am NIC
-   src 192.168.2.10 → ausgehende prim. Adr.
-   via → zeigt auf Router eines Netzes und NIC default → default Route

#### Routing setzen/löschen

```bash
# ip add 192.168.1.0/24 via 192.168.2.2
# ip add 192.168.3.0/24 dev eth1 via 192.168.2.2
# ip route delete 192.168.2.0/24
```
-   Ändern der Routingtabelle immer mit
```bash
# ip route flush cache
```

Den Cache vom Routing leeren!

#### Advanced Routing

-   Routing Entscheidungen anhand von Quelladressen, Eingangsinterface
    oder iptabels treffen. → Policy Routing
-   Policy Routing Database (PRDB) speichern Policy Routing Tables

In [77]:
ip rule show

0:	from all lookup local 
32766:	from all lookup main 
32767:	from all lookup default 


-   local wird vom Kernel verwaltet (hohe Prio)
-   main wird gesetzt von ip route bzw. route. Ist die Standardtabelle.
-   default standardmäßig leer

Zeigt die Tabelle local an

In [78]:
sudo ip route show table local

broadcast 10.0.2.0 dev enp0s3  proto kernel  scope link  src 10.0.2.15 
local 10.0.2.15 dev enp0s3  proto kernel  scope host  src 10.0.2.15 
broadcast 10.0.2.255 dev enp0s3  proto kernel  scope link  src 10.0.2.15 
broadcast 10.0.3.0 dev lxcbr0  proto kernel  scope link  src 10.0.3.1 
local 10.0.3.1 dev lxcbr0  proto kernel  scope host  src 10.0.3.1 
broadcast 10.0.3.255 dev lxcbr0  proto kernel  scope link  src 10.0.3.1 
broadcast 127.0.0.0 dev lo  proto kernel  scope link  src 127.0.0.1 
local 127.0.0.0/8 dev lo  proto kernel  scope host  src 127.0.0.1 
local 127.0.0.1 dev lo  proto kernel  scope host  src 127.0.0.1 
broadcast 127.255.255.255 dev lo  proto kernel  scope link  src 127.0.0.1 
broadcast 172.17.0.0 dev docker0  proto kernel  scope link  src 172.17.0.1 linkdown 
local 172.17.0.1 dev docker0  proto kernel  scope host  src 172.17.0.1 
broadcast 172.17.255.255 dev docker0  proto kernel  scope link  src 172.17.0.1 linkdown 
broadcast 172.22.3.0 dev enp0s8  proto kernel  scope

#### Neue Routing Tabelle anlegen

In [79]:
cat /etc/iproute2/rt_tables

#
# reserved values
#
255	local
254	main
253	default
0	unspec
#
# local
#
#1	inr.ruhep


Nummern 0 bis 255 und die jeweiligen Namen dazu

In [82]:
echo "100 table_in_enp0s3" | sudo tee -a /etc/iproute2/rt_tables

100 table_in_enp0s3


In [86]:
sudo ip rule add iif enp0s3 table table_in_enp0s3 priority 10000

In [87]:
cat /etc/iproute2/rt_tables

#
# reserved values
#
255	local
254	main
253	default
0	unspec
#
# local
#
#1	inr.ruhep
100 table_in_enp0s3


In [88]:
sudo ip rule show

0:	from all lookup local 
10000:	from all iif enp0s3 lookup table_in_enp0s3 
32766:	from all lookup main 
32767:	from all lookup default 


In [89]:
sudo ip rule delete priority 10000

In [90]:
sudo ip rule show

0:	from all lookup local 
32766:	from all lookup main 
32767:	from all lookup default 


Routing mehrere Uplinks

-   \# echo „100 provider\_B“ &gt;&gt; /etc/iproute2/rt\_tables

### ARP

In [71]:
ip neighbour show

10.0.2.2 dev enp0s3 lladdr 52:54:00:12:35:02 STALE
10.0.3.84 dev lxcbr0 lladdr 00:16:3e:49:df:7c STALE
10.0.3.66 dev lxcbr0 lladdr 00:16:3e:ba:f2:49 STALE
10.0.3.166 dev lxcbr0 lladdr 00:16:3e:9e:3d:b5 STALE
10.0.3.65 dev lxcbr0 lladdr 00:16:3e:0a:e3:13 STALE
10.0.3.30 dev lxcbr0 lladdr 00:16:3e:1f:55:4d STALE
172.22.3.1 dev enp0s8 lladdr 0a:00:27:00:00:00 DELAY
10.0.3.230 dev lxcbr0 lladdr 00:16:3e:ca:f9:07 STALE
10.0.3.120 dev lxcbr0 lladdr 00:16:3e:83:c5:09 STALE


#### ARP Referenz

- \# ip neighbour show  
  \# ip n s
- \# ip neighbour delete 192.168.2.1 dev eth0  
- \# ip neighbour change 192.168.99.2 dev eth0 \\ lladdr
    00:23:08:f9:81:ac nud permanent nud für Neighbour Unreachability
    Detection

In [72]:
ip n s

10.0.2.2 dev enp0s3 lladdr 52:54:00:12:35:02 STALE
10.0.3.84 dev lxcbr0 lladdr 00:16:3e:49:df:7c STALE
10.0.3.66 dev lxcbr0 lladdr 00:16:3e:ba:f2:49 STALE
10.0.3.166 dev lxcbr0 lladdr 00:16:3e:9e:3d:b5 STALE
10.0.3.65 dev lxcbr0 lladdr 00:16:3e:0a:e3:13 STALE
10.0.3.30 dev lxcbr0 lladdr 00:16:3e:1f:55:4d STALE
172.22.3.1 dev enp0s8 lladdr 0a:00:27:00:00:00 REACHABLE
10.0.3.230 dev lxcbr0 lladdr 00:16:3e:ca:f9:07 STALE
10.0.3.120 dev lxcbr0 lladdr 00:16:3e:83:c5:09 STALE


In [74]:
sudo ip neighbour delete 10.0.3.84 dev lxcbr0

In [75]:
ip n s

10.0.2.2 dev enp0s3 lladdr 52:54:00:12:35:02 STALE
10.0.3.84 dev lxcbr0  FAILED
10.0.3.66 dev lxcbr0 lladdr 00:16:3e:ba:f2:49 STALE
10.0.3.166 dev lxcbr0 lladdr 00:16:3e:9e:3d:b5 DELAY
10.0.3.65 dev lxcbr0 lladdr 00:16:3e:0a:e3:13 STALE
10.0.3.30 dev lxcbr0 lladdr 00:16:3e:1f:55:4d STALE
172.22.3.1 dev enp0s8 lladdr 0a:00:27:00:00:00 REACHABLE
10.0.3.230 dev lxcbr0 lladdr 00:16:3e:ca:f9:07 STALE
10.0.3.120 dev lxcbr0 lladdr 00:16:3e:83:c5:09 STALE


## Übertragungsprotokolle

Kommunikationsprotokolle

Anwendungsprotokolle

-   SMTP (für E-Mail),
-   SSH (für interaktive Sitzungen auf entfernten Rechnern),
-   DNS (für die Auflösung von Rechnernamen) oder
-   SIP (für Internet-Telefonie) ...

## Grundlegendes

* Loopback-Adresse (127.0.0.1)
* eindeutige IP-Adresse pro Netzwerkschnittstelle
* Routing (KNT)
    * Default Route
    * NAT ...
* IP-Adressen zu Namen herausfinden (DNS)

### Linux-Rechner Netzwerkesseintals

Variante 1 (statisch)

-   Eigene IP-Adresse
-   Default Gateway
-   Subnetzmaske
-   Adresse(n) DNS

Variante 2 (dynamisch)

-   DHCP

DNS im eigenen Netzwerk

### Netzwerk DNS

1. Root-Level-Nameserver
    -   Punkt am Schluss
2. Nameserver für TLD wie .de
3. Nameserver für ein Netz wie irgengwas.de.

## Übungen

- Welche Transportprotokolle (außer Ethernet und IEEE-802.11) kennen Sie? Welche Kommunikationsprotokolle (außer IP, TCP und UDP)?  
  Welche Anwendungsprotokolle außer den im Text genannten?
- Wie viele nutzbare IP-Adressen enthält das Netz 10.11.12.0/22?  
  (Ziehen Sie wegen deren Sonderbedeutung die erste und letzte Adresse ab.)
- Welche Voraussetzungen müssen Sie erfüllen, um einen Domainnamen in der de-Top-Level-Domain registrieren zu können?

## Netzwerk Konfigurationsdateien

Debian GNU
- /etc/network/interfaces  
  https://wiki.ubuntuusers.de/interfaces/
- /usr/share/doc/ifupdown/examples/networkinterfaces.gz
- Portnummern werden unter /etc/services aufgeführt

SUSE
- /etc/sysconfig/network

Red-Hat
- /etc/sysconfig/network- scripts

### Flüchtige Einstellungen

Für kurzzeitige Experimente ohne Eintrag in Konfigurationsdatei.

```bash
ifconfig eth0 192.168.178.111 netmask 255.255.255.0
route add -net default gw 192.168.178.1
```
Mit iproute2 (IPv4 und IPv6)

```bash
ip addr set 192.168.178.111/24 broadcast 192.168.178.255 dev eth0
ip route add default via 192.168.178.1 dev eth0
```
  
Siehe auch: http://wiki.ubuntuusers.de/ip

### DNS Eintrag

Die IP-Adressen von DNS-Servern (Achtung DHCP)

Ab Ubuntu 12.04 Server `resolver` installiert

In [91]:
cat /etc/resolv.conf

# Dynamic resolv.conf(5) file for glibc resolver(3) generated by resolvconf(8)
#     DO NOT EDIT THIS FILE BY HAND -- YOUR CHANGES WILL BE OVERWRITTEN
nameserver 10.0.2.3


## Dauerhafte Einstellung

In [92]:
cat /etc/network/interfaces

# This file describes the network interfaces available on your system
# and how to activate them. For more information, see interfaces(5).

source /etc/network/interfaces.d/*

# The loopback network interface
auto lo
iface lo inet loopback

# The primary network interface
auto enp0s3
iface enp0s3 inet dhcp
pre-up sleep 2
#VAGRANT-BEGIN
# The contents below are automatically generated by Vagrant. Do not modify.
auto enp0s8
iface enp0s8 inet static
      address 172.22.3.2
      netmask 255.255.255.0
#VAGRANT-END


## Netzwerk Fehlersuche

- Das UP beachten
- Loopback Device anpingen
- Ping auf die Adresse der Netzwerkkarte
- Default Gateway erreichbar
- Route richtig eingestellt
- Meist HW-Fehler!
- Schnelles pingen mit der Option -f (root darf schneller)
- Das Kommando dig dient zum Testen der Namensauflösung via DNS.

In [122]:
dig www.heise.de


; <<>> DiG 9.10.3-P4-Ubuntu <<>> www.heise.de
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 40536
;; flags: qr rd ra; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 0

;; QUESTION SECTION:
;www.heise.de.			IN	A

;; ANSWER SECTION:
www.heise.de.		3600	IN	A	193.99.144.85

;; Query time: 0 msec
;; SERVER: 10.0.2.3#53(10.0.2.3)
;; WHEN: Mon Feb 19 20:30:13 UTC 2018
;; MSG SIZE  rcvd: 46



In [97]:
# dig www.heise.de @fritz.box
# dig www.heise.de @192.168.99.117

In [121]:
dig franken.de mx


; <<>> DiG 9.10.3-P4-Ubuntu <<>> franken.de mx
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOTIMP, id: 39106
;; flags: qr rd ra; QUERY: 1, ANSWER: 0, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 4096
;; QUESTION SECTION:
;franken.de.			IN	MX

;; Query time: 0 msec
;; SERVER: 10.0.2.3#53(10.0.2.3)
;; WHEN: Mon Feb 19 20:29:58 UTC 2018
;; MSG SIZE  rcvd: 39



In [101]:
dig +short -x 193.99.144.85

www.heise.de.


- Das Kommando netstat ist eine Art Schweizer Messer, das Ihnen Informationen aller Art über Ihren Rechner und seine Netzanbindung liefert.

In [120]:
netstat #Aktive Verbindungen

Aktive Internetverbindungen (ohne Server)
Proto Recv-Q Send-Q Local Address           Foreign Address         State      
tcp        0      0 localhost:57878         localhost:38151         TIME_WAIT  
tcp        0      0 localhost:52378         localhost:39889         VERBUNDEN  
tcp        0      0 localhost:57860         localhost:38151         VERBUNDEN  
tcp        0      0 localhost:39889         localhost:51010         VERBUNDEN  
tcp        0      0 localhost:49654         localhost:59652         VERBUNDEN  
tcp        0      0 172.22.3.2:8888         michl-laptop:42416      VERBUNDEN  
tcp        0      0 localhost:57884         localhost:38151         VERBUNDEN  
tcp        0      0 172.22.3.2:8888         michl-laptop:42452      VERBUNDEN  
tcp        0      0 localhost:54382         localhost:56932         TIME_WAIT  
tcp        0      0 172.22.3.2:8888         michl-laptop:42432      VERBUNDEN  
tcp        0      0 localhost:59652         localhost:49654         VERBUNDEN 

limit_output extension: Maximum message size of 1000 exceeded with 1002 characters

- Wichtige Optionen: -u; -n; -s; -r; -p (nur root)

In [123]:
netstat -tl #Aktive TCP-Verbindungen

Aktive Internetverbindungen (Nur Server)
Proto Recv-Q Send-Q Local Address           Foreign Address         State      
tcp        0      0 *:ssh                   *:*                     LISTEN     
tcp        0      0 localhost:42999         *:*                     LISTEN     
tcp        0      0 *:8888                  *:*                     LISTEN     
tcp        0      0 localhost:48122         *:*                     LISTEN     
tcp        0      0 *:51452                 *:*                     LISTEN     
tcp        0      0 *:33277                 *:*                     LISTEN     
tcp        0      0 *:50719                 *:*                     LISTEN     
tcp        0      0 localhost:49760         *:*                     LISTEN     
tcp        0      0 *:nfs                   *:*                     LISTEN     
tcp        0      0 localhost:56932         *:*                     LISTEN     
tcp        0      0 localhost:59652         *:*                     LISTEN     

limit_output extension: Maximum message size of 1000 exceeded with 1001 characters

In [124]:
netstat -s

Ip:
    146991 total packets received
    2 with invalid addresses
    0 forwarded
    0 incoming packets discarded
    146960 incoming packets delivered
    112500 requests sent out
    53 dropped because of missing route
Icmp:
    7 ICMP messages received
    0 input ICMP message failed.
    ICMP-Eingabehistogramm:
        destination unreachable: 1
        echo replies: 6
    16 ICMP messages sent
    0 ICMP messages failed
    ICMP-Ausgabehistogramm:
        destination unreachable: 3
        echo request: 13
IcmpMsg:
        InType0: 6
        InType3: 1
        OutType3: 3
        OutType8: 13
Tcp:
    2293 active connections openings
    1735 passive connection openings
    179 failed connection attempts
    27 connection resets received
    30 connections established
    145160 segments received
    120849 segments send out
    18 segments retransmited
    0 bad segments received.
    198 resets sent
Udp:
    1711 packets received
    2 packets to unknown port received.
    0 p

limit_output extension: Maximum message size of 1000 exceeded with 1021 characters

In [125]:
netstat -r

Kernel-IP-Routentabelle
Ziel            Router          Genmask         Flags   MSS Fenster irtt Iface
10.0.2.0        *               255.255.255.0   U         0 0          0 enp0s3
10.0.3.0        *               255.255.255.0   U         0 0          0 lxcbr0
172.17.0.0      *               255.255.0.0     U         0 0          0 docker0
172.22.3.0      *               255.255.255.0   U         0 0          0 enp0s8
192.168.122.0   *               255.255.255.0   U         0 0          0 virbr0


In [126]:
sudo netstat -p

Aktive Internetverbindungen (ohne Server)
Proto Recv-Q Send-Q Local Address           Foreign Address         State       PID/Program name
tcp        0      0 localhost:57878         localhost:38151         TIME_WAIT   -               
tcp        0      0 localhost:52378         localhost:39889         VERBUNDEN   1065/python3    
tcp        0      0 localhost:57860         localhost:38151         VERBUNDEN   1065/python3    
tcp        0      0 localhost:39889         localhost:51010         VERBUNDEN   3432/python3    
tcp        0      0 localhost:49654         localhost:59652         VERBUNDEN   1065/python3    
tcp        0      0 172.22.3.2:8888         michl-laptop:42416      VERBUNDEN   1065/python3    
tcp        0      0 localhost:57884         localhost:38151         VERBUNDEN   1065/python3    
tcp        0      0 172.22.3.2:8888         michl-laptop:42452      VERBUNDEN   1065/python3    
tcp        0      0 localhost:54382         localhost:56932         TIME_WAIT   -    

limit_output extension: Maximum message size of 1000 exceeded with 1012 characters

- Alternativ mit ip `ss` (show socket)

In [127]:
ss

Netid  State      Recv-Q Send-Q Local Address:Port                 Peer Address:Port                
u_str  ESTAB      0      0      @/tmp/dbus-rjkrYOdJqw 26776                 * 26774                
u_str  ESTAB      0      0       * 25278                 * 25279                
u_str  ESTAB      0      0      @/com/ubuntu/upstart-session/900/2252 25805                 * 25802                
u_str  ESTAB      0      0       * 14880                 * 14306                
u_str  ESTAB      0      0       * 26947                 * 26948                
u_str  ESTAB      0      0      /var/run/dbus/system_bus_socket 22806                 * 22419                
u_str  ESTAB      0      0       * 22078                 * 22077                
u_str  ESTAB      0      0       * 1073098               * 1073099              
u_str  ESTAB      0      0       * 28842                 * 28843                
u_str  ESTAB      0      0      /var/run/dbus/system_bus_socket 16160                 *

limit_output extension: Maximum message size of 1000 exceeded with 1023 characters

## Übung

- Verwenden Sie ping, um sich zu vergewissern, dass Sie einen wohlbekannten Rechner im Internet erreichen können (vielleicht www.google.com).
- Benutzen Sie dig, um nachzuschauen, für welche IP-Adresse www.heise.de steht.
- Die Option +trace von dig bringt das Programm dazu, die komplette Suchkette für einen Namen zu dokumentieren, beginnend bei den Root-Level-Nameservern. Probieren Sie das für einige interessante Namen aus und prüfen Sie die Zwischenschritte.
- Welche Netzwerkdienste bietet Ihr Rechner an? Welche davon sind von anderen Rechnern aus erreichbar?

## Kommandos

- dig Sucht Informationen im DNS (sehr komfortabel) dig(1)
- ifconfig Konfiguriert Netzwerk-Schnittstellen ifconfig (8)
- ip Konfiguriert Netzwerk-Schnittstellen
- netstat Liefert Informationen über Netzverbindungen, Server, Routing netstat(8)
- ping Prüft grundlegende Netzwerk-Konnektivität mit ICMP ping(8)
- ping6 Prüft grundlegende Netzwerk-Konnektivität (für IPv6) ping(8)
- route Verwaltet die statische Routing-Tabelle im Linux-Kern route(8)
- ss wie netstat show socket

## Zusammenfassung

- Es gibt drei Arten von Netzwerkprotokollen: Transportprotokolle, Kommunikationsprotokolle und Anwendungsprotokolle.
- In einem »Protokollstapel« verkehrt jedes Protokoll nur mit den unmittelbar darunter- und darüberliegenden  Protokollen.
- TCP/IP enthält die Kommunikationsprotokolle IP, TCP (zuverlässig und verbindungsorientiert) und UDP (unzuverlässig und verbindungslos), das Steuerungsprotokoll ICMP und eine Vielzahl von Anwendungsprotokollen auf TCP- oder UDP-Basis.
- Rechner im Internet haben eindeutige IP-Adressen.
- Routing erlaubt die Kommunikation zwischen nicht direkt verbundenen Rechnern.
-   Um am Netzwerk teilzunehmen, benötigt ein Linux-Rechner eine IP-Adresse, eine Subnetzadresse mit Maske, ein Default-Gateway und die Adresse mindestens eines DNS-Servers.
- Das DNS ist eine verteilte Datenbank zur Abbildung von Rechnernamen auf IP-Adressen und umgekehrt (unter anderem).
- IPv6 ist der Nachfolgestandard von IPv4, der verschiedene Einschränkungen aufhebt und Verbesserungen enthält.
- ifconfig und route dienen zur manuellen Konfiguration der Netzanbindung. Distributionen haben verschiedene Methoden zur dauerhaften Netzkonfiguration.
- Die Programme ifconfig, ping, dig und netstat dienen zur Fehlersuche im Netz. Das Programm ip ersetzt ifconfig und netstat.

## Anhang

Verz: /proc/net und /proc/sys/net

- Standardeinstellungen von Netzwerkkarten Protokollen und Modulen.
- Nur ändern wenn Ahnung! Besser über geeignete Kommandos ändern.

In [128]:
cat /proc/sys/net/ipv4/ip_forward #Aktivierung von IPv4 Routing

1


In [129]:
ls /proc/sys/net/ipv4/conf #Einstellungen zu den Netzwerkkarten

all      docker0  enp0s8  lxcbr0      vethBOLXNY  virbr0      vnet0
default  enp0s3   lo      veth2QYQL4  vethDAI3BY  virbr0-nic


In [130]:
ls /proc/sys/net/ipv4/conf/enp0s8

accept_local         disable_xfrm                        proxy_arp
accept_redirects     force_igmp_version                  proxy_arp_pvlan
accept_source_route  forwarding                          route_localnet
arp_accept           igmpv2_unsolicited_report_interval  rp_filter
arp_announce         igmpv3_unsolicited_report_interval  secure_redirects
arp_filter           ignore_routes_with_linkdown         send_redirects
arp_ignore           log_martians                        shared_media
arp_notify           mc_forwarding                       src_valid_mark
bootp_relay          medium_id                           tag
disable_policy       promote_secondaries


Änderungen in /proc sind nicht dauerhaft.

Dauerhafte Änderungen in /etc/sysctl.conf

- Syntax: /proc/sys/net/ipv4/ip\_forwarding wird zu net.ipv4.ip\_forwarding=1
- Übernahme ohne reboot mit \# sysctl -p

In [131]:
cat /etc/sysctl.conf

#
# /etc/sysctl.conf - Configuration file for setting system variables
# See /etc/sysctl.d/ for additional system variables.
# See sysctl.conf (5) for information.
#

#kernel.domainname = example.com

# Uncomment the following to stop low-level messages on console
#kernel.printk = 3 4 1 3

##############################################################3
# Functions previously found in netbase
#

# Uncomment the next two lines to enable Spoof protection (reverse-path filter)
# Turn on Source Address Verification in all interfaces to
# prevent some spoofing attacks
#net.ipv4.conf.default.rp_filter=1
#net.ipv4.conf.all.rp_filter=1

# Uncomment the next line to enable TCP/IP SYN cookies
# See http://lwn.net/Articles/277146/
# Note: This may impact IPv6 TCP sessions too
#net.ipv4.tcp_syncookies=1

# Uncomment the next line to enable packet forwarding for IPv4
#net.ipv4.ip_forward=1

# Uncomment the next line to enable packet forwarding for IPv6
#  Enabling this option disables Stateless Addr

limit_output extension: Maximum message size of 1000 exceeded with 1022 characters